In [8]:
from pytezos import pytezos

from config import KEY, SHELL_URI, JUSTER_ADDRESS
client = pytezos.using(key=KEY, shell=SHELL_URI)
contract = client.contract(JUSTER_ADDRESS)

In [ ]:
from run_maker import generate_event_lines
generate_event_lines()

In [9]:
from dipdup import JusterDipDupClient
from executors import WithdrawCaller
import time
from utility import timestamp_to_date

dd_client = JusterDipDupClient()
REWARD_SPLIT_FEE_AFTER = 24*60*60
closed_before = timestamp_to_date(time.time() - REWARD_SPLIT_FEE_AFTER)
withdrawable_events = dd_client.query_withdrawable_events(closed_before)

In [11]:
# withdrawable_events

In [18]:
contract.storage['events'][7]()

KeyError: 7

### Why 7 and 12 events have unwithdrawn positions?

In [133]:
import requests
import json
import pandas as pd
from urllib.parse import urlencode

method = f'https://api.florencenet.tzkt.io/v1/accounts/{JUSTER_ADDRESS}/operations?'
params = urlencode({
    'limit': 1000,
    'parameter.eventId.eq': 3,
    # 'parameters': 'withdraw'
})
response = requests.get(method + params)

data = json.loads(response.text)
df = pd.json_normalize(data)
len(df)

8

In [134]:
df[['parameter.entrypoint']]

,parameter.entrypoint
0,NaN
1,withdraw
2,withdraw
3,withdraw
4,withdraw
5,withdraw
6,provideLiquidity
7,NaN


In [135]:
withdrawals = df[ df['parameter.entrypoint'] == 'withdraw' ]
withdrawals

,type,id,level,timestamp,block,hash,counter,nonce,gasLimit,gasUsed,...,parameter.value.eventId,parameter.value.participantAddress,parameter.value.maxSlippage,parameter.value.expectedRatioBelow,parameter.value.expectedRatioAboveEq,contractBalance,originatedContract.kind,originatedContract.address,originatedContract.typeHash,originatedContract.codeHash
1,transaction,3132285,284963,2021-06-22T11:28:30Z,BLMwhz7Hi2Tg4wyWXVYP3xjGSB9aeV26RCS5zTm6AmY5gp...,opNJ7tLmtAAJQSyJNa2XrSmp6u9GFrRC5KthvCmgdvjCqn...,52012,NaN,135035,133508,...,3,tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,transaction,3132284,284963,2021-06-22T11:28:30Z,BLMwhz7Hi2Tg4wyWXVYP3xjGSB9aeV26RCS5zTm6AmY5gp...,opQ5hV9c4bbQrzCotAAGrx7GscvJSe4TarNZVAp4QsPtc1...,52029,NaN,132482,132382,...,3,tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,transaction,3132056,284950,2021-06-22T11:21:40Z,BLaG3qFCcuv3LUgbCnHErrqc8TwSqd4LxkFHcVBEPFAYwo...,opHhUbwQELo9kjbi3E9JA2uWACX8kUFTRB3Kiu2F7wwyn5...,52013,NaN,132482,132382,...,3,tz1ZAzDvkZCT2LAyPN8Kdxw3kes7xfWerZhZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,transaction,3131994,284946,2021-06-22T11:19:40Z,BKzpq4jz6DvU4d91CchU32f2cXM3ob5g4GEPpKfgZxRpW9...,oordx4N4zVkNqeZDCJk2D1Zcyr4jjGtwbc6PKiKiG85csb...,52019,NaN,132482,132382,...,3,tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,transaction,3131930,284942,2021-06-22T11:17:40Z,BLWBVexAw1YFmaBbUoxFPdwzEAwSHacVJE4ztRP4oRsoNF...,ooE2PBZQ9y9H2E2Zs8C6s1C6DNENiKhXj8D7eZJJgGCu2b...,52021,NaN,132482,132382,...,3,tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
withdrawals.hash.to_dict()

{1: 'oo5gGoR3NnnnZBgt4FJw6EFz3bN1QqdRTFnvCQgGhnfdFnPZdCe',
 2: 'opF9uh7hbgd7wzmp7HYQ6vwVvrYaQwFhB9syyeCgnkeU15B8jiG'}

In [107]:
withdrawals.iloc[0]['hash'].to_dict()

type                                                                          transaction
id                                                                                3436267
level                                                                              307831
timestamp                                                            2021-06-30T15:39:34Z
block                                   BMQSErNSwsXvZrecCmAKBSgP62vZDtqvUY5f9Rqgyp15j7...
hash                                    oo5gGoR3NnnnZBgt4FJw6EFz3bN1QqdRTFnvCQgGhnfdFn...
counter                                                                             52058
nonce                                                                                 NaN
gasLimit                                                                           137452
gasUsed                                                                            134498
bakerFee                                                                            14093
amount    

In [68]:
# https://better-call.dev/florencenet/opg/opF9uh7hbgd7wzmp7HYQ6vwVvrYaQwFhB9syyeCgnkeU15B8jiG/contents
# participants 2 -> 1

w12.iloc[0].hash

'opF9uh7hbgd7wzmp7HYQ6vwVvrYaQwFhB9syyeCgnkeU15B8jiG'

### Juster-Maker:
Tool allowing to run events in cycle

### Создание линеек ивентов:
- Линейка ивентов - это набор из однотипных ивентов которые создаются друг за другом с наложением периода принятия ставок и периода измерения.
- Тип ивента может быть определён по 3 параметрам:
    - валютная пара
    - target dynamics
    - period
- Также ивенты могут отличаются liquidity_percent но сейчас я считаю что liquidity percent привязыватеся к period + target_dynamics

- TODO: вынести в отдельный ipynb или отдельный скрипт (если отдельный скрипт - хорошо было бы описать этот процесс в readme.MD, либо может нужно вообще CLI утилиту сделать в которой одна из точек входа это генерация линеек)

### TODO: remake all using apscheduler
    https://apscheduler.readthedocs.io/en/stable/

### TODO: Autowithdrawal after 24 hours
    - for all events check if it is closed for more than 24 hours, if it is, make withdrawal transactions for all participants

### TODO: add here pusher & startMeasurement/close?
### TODO: in event creation somehow check that event successfully created and if it is not: make another in 5 minutes?

### TODO: it is better not to return transactions in pool by BulkSigner, but reemmit all transactions by EventEmmitter / LiquidityProvider instead
- so EventEmmitter can just check either transaction was successfull or not (after some pause for 10 mins for example) and only add period to timer if it was succeed
- the same with LiquidityProvider who just checks only for the last possible events in line, so if it is new event: he would use it

### ++ TODO: PROVIDE LIQUIDITY for this events

### ++ TODO: OK, в общем проще всего мне кажется вот так:
    - для каждой линейки ивентов создаётся свой event_emitter, он проверяет что пришло время запустить ивет сравнивая текущее значение таймера с таргетом. И после запуска ивента он переставляет таймер дальше.
    - с определённой частотой (например раз в 20 секунд) таймер проверяется и если время больше чем в таймере - транза с новым ивентом закидывается в очередь и подхватывается BulkSender

### TODO: maybe make next_event func/iterator that returns time and params for the next events

### TODO: need to have some tests for LoopExecutor:
- for example that it runned function K times in given time

### TODO: need to test all this code before running in florencenet

### TODO: provide liquidity
### TODO: pushing prices